<a href="https://colab.research.google.com/github/lsgotti/TCC_Python/blob/main/Preprocessamento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#carrega imports e instala o scikit-multiflow
!pip install -U scikit-multiflow
# Imports
from skmultiflow.data import FileStream
from skmultiflow.data import SEAGenerator
from skmultiflow.evaluation import EvaluatePrequential
from skmultiflow.bayes import NaiveBayes
from skmultiflow.trees import HoeffdingTreeClassifier
from skmultiflow.trees import HoeffdingAdaptiveTreeClassifier
from skmultiflow.trees import ExtremelyFastDecisionTreeClassifier
from skmultiflow.meta import LearnPPNSEClassifier
from skmultiflow.drift_detection import ADWIN
from skmultiflow.meta import OzaBaggingClassifier

from sklearn.linear_model import SGDClassifier

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec

#carrega imports
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import svm
import pandas as pd
from sklearn import datasets, linear_model
from matplotlib import pyplot as plt
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix

from google.colab import files

import numpy as np
from skmultiflow.data.file_stream import FileStream
from skmultiflow.data.data_stream import DataStream
from skmultiflow.trees import HoeffdingTreeClassifier
from skmultiflow.evaluation import EvaluatePrequential
import pandas as pd
import io
from google.colab import files
from skmultiflow.trees import HoeffdingTree
from datetime import datetime, date

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 7.6 MB/s 


In [ ]:
#upload do arquivo para gerar base
uploaded = files.upload()
#Importa o arquivo para o colab, é preciso colocar o nome do arquivo
import io
df = pd.read_csv(io.BytesIO(uploaded['all_projects_with_dates_new.csv']))

Saving all_projects_with_dates_new.csv to all_projects_with_dates_new.csv


In [ ]:
#gerar base
print(df.shape)
#Colocar o w. w = numero de dias para confirmar a classe da instancia x86400
w = 100 * 86400
#remove instancias marcadas com codigos 2 e 3 (se alguma)
df2= df.drop(df[(df['code'] == 2) | (df['code'] == 3)].index)
#transforma o dataframe em numpy array
df2_np = df2.to_numpy()
#lista onde ficarão as instancias replicadas
mylist = []
df2_np

for n in range(df2_np.shape[0]):
  #se a classe for 0 (posição [n][2])
  if(df2_np[n][15]==0):
    #Coloca o codigo 0
    df2_np[n][19]=0
    #cria uma nova instancia copiando a anterior
    new_row=np.copy(df2_np[n])
    #coloca classe 0
    new_row[15]=0
    #adiciona w ao timestamp atual
    new_row[0]=new_row[16] + w
    #coloca o codigo como 1
    new_row[19]=1
    #adiciona instancia a lista
    mylist.append(new_row)
  #se a classe for 1 (posição [n][2])
  if(df2_np[n][15]==1):
    #Coloca o codigo 0
    df2_np[n][19]=0
    #cria uma nova instancia copiando a anterior
    new_row=np.copy(df2_np[n])
    #coloca classe 1
    new_row[15]=1
    #adiciona days_to_fix (posição [17]) ao timestamp atual
    new_row[0]=new_row[16] + (new_row[18] * 86400)
    #coloca o codigo como 1
    new_row[19]=1
    #adiciona instancia a lista
    mylist.append(new_row)
    #se days_to_fix x 86400 (converção para timestamp) for maior do que o tempo w
    if(df2_np[n][18] * 86400 > w):
      #cria uma nova instancia copiando a anterior
      new_row=np.copy(df2_np[n])
      #coloca classe 0
      new_row[15]=0
      #adiciona w ao timestamp atual
      new_row[0]=new_row[0] + w
      #coloca o codigo como 1
      new_row[19]=1
      #adiciona instancia a lista
      mylist.append(new_row)
newInstances = np.array(mylist)

#junta as instancias novas as antigas
df2_np = np.vstack((df2_np,newInstances))
df2_np

#gerar dataframe e baixar como csv
new_df = pd.DataFrame(
    {'author_date_unix_timestamp': df2_np[:, 0],
     'fix':  df2_np[:, 1],
     'ns':  df2_np[:, 2],
     'nd':  df2_np[:, 3],
     'nf':  df2_np[:, 4],
     'entrophy':  df2_np[:, 5],
     'la':  df2_np[:, 6],
     'ld':  df2_np[:, 7],
     'lt':  df2_np[:, 8],
     'ndev':  df2_np[:, 9],
     'age':  df2_np[:, 10],
     'nuc':  df2_np[:, 11],
     'exp':  df2_np[:, 12],
     'rexp':  df2_np[:, 13],
     'sexp':  df2_np[:, 14],
     'contains_bug':  df2_np[:, 15],
     'original_timestamp':  df2_np[:, 16],
     'repository_name':  df2_np[:, 17],
     'days_to_first_fix':  df2_np[:, 18],
     'code':  df2_np[:, 19]
    })

new_df.sort_values(by=['author_date_unix_timestamp'])
new_df
new_df.to_csv('all_projects_with_dates_replicated.csv')
files.download('all_projects_with_dates_replicated.csv')

(197475, 20)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#gerar base
print(df.shape)
#Colocar o w. w = numero de dias para confirmar a classe da instancia x86400
w = 100 * 86400
#remove instancias marcadas com codigos 2 e 3 (se alguma)
df2= df.drop(df[(df['code'] == 2) | (df['code'] == 3)].index)
#transforma o dataframe em numpy array
df2_np = df2.to_numpy()
#lista onde ficarão as instancias replicadas
mylist = []
df2_np

for n in range(df2_np.shape[0]):
  #se a classe for 0 (posição [n][2])
  if(df2_np[n][2]==0):
    #Coloca o codigo 0
    df2_np[n][20]=0
    #cria uma nova instancia copiando a anterior
    new_row=np.copy(df2_np[n])
    #coloca classe 0
    new_row[2]=0
    #adiciona w ao timestamp atual
    new_row[19]=new_row[0] + w
    #coloca o codigo como 1
    new_row[20]=1
    #adiciona instancia a lista
    mylist.append(new_row)
  #se a classe for 1 (posição [n][2])
  if(df2_np[n][2]==1):
    #Coloca o codigo 0
    df2_np[n][20]=0
    #cria uma nova instancia copiando a anterior
    new_row=np.copy(df2_np[n])
    #coloca classe 1
    new_row[2]=1
    #adiciona days_to_fix (posição [17]) ao timestamp atual
    new_row[19]=new_row[0] + (new_row[17] * 86400)
    #coloca o codigo como 1
    new_row[20]=1
    #adiciona instancia a lista
    mylist.append(new_row)
    #se days_to_fix x 86400 (converção para timestamp) for maior do que o tempo w
    if(df2_np[n][17] * 86400 > w):
      #cria uma nova instancia copiando a anterior
      new_row=np.copy(df2_np[n])
      #coloca classe 0
      new_row[2]=0
      #adiciona w ao timestamp atual
      new_row[19]=new_row[0] + w
      #coloca o codigo como 1
      new_row[20]=1
      #adiciona instancia a lista
      mylist.append(new_row)
newInstances = np.array(mylist)

#junta as instancias novas as antigas
df2_np = np.vstack((df2_np,newInstances))
df2_np

#gerar dataframe e baixar como csv
new_df = pd.DataFrame(
    {'author_date_unix_timestamp': df2_np[:, 0],
     'fix':  df2_np[:, 1],
     'contains_bug':  df2_np[:, 2],
     'ns':  df2_np[:, 3],
     'nd':  df2_np[:, 4],
     'nf':  df2_np[:, 5],
     'entrophy':  df2_np[:, 6],
     'la':  df2_np[:, 7],
     'ld':  df2_np[:, 8],
     'lt':  df2_np[:, 9],
     'ndev':  df2_np[:, 10],
     'age':  df2_np[:, 11],
     'nuc':  df2_np[:, 12],
     'exp':  df2_np[:, 13],
     'rexp':  df2_np[:, 14],
     'sexp':  df2_np[:, 15],
     'Unnamed: 16':  df2_np[:, 16],
     'days_to_first_fix':  df2_np[:, 17],
     'date':  df2_np[:, 18],
     'original_timestamp':  df2_np[:, 19],
     'code':  df2_np[:, 20]
    })

new_df.sort_values(by=['author_date_unix_timestamp'])
new_df
new_df.to_csv('spring-integration2.csv')
files.download('spring-integration2.csv')